In this notebook the historical values of the fireindices are calculated and saved in a file. To calculate new ones, you can directly download the historical data from the daggegevens website in csv format:
https://daggegevens.knmi.nl/klimatologie/daggegevens
with one station of the stations and the fields(or velden) FG, TX, RH and UG ticked.

In [1]:
from FireWeatherIndex import FWICLASS
import matplotlib.pyplot as plt
import csv
import pandas as pd

Here you can put the filepath of the downloaded file. To in the end calculate the values of the latest dates if possible. For this all the given ticked functions have to be there.

In [2]:
#Inputs:
file_path = 'Guido-data/Dagdata/result (58).txt'

In here we open the file and put in into tha data that is used. After that we find the station code to create the file numbers, automatically.

In [3]:
#read CSV from a .txt file


data=[]

with open(file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        data.append(row)

data=data[12:]

station_code=data[0][0]
station_code=int(station_code)
station_code=str(station_code)
print(station_code)

260


0=station
1=date
2=windspeed
3=temperature
4=neerslag
5=luchtvochtigheid

Below we find the first possible data entry where all 4 of the data constraints are met. For example in De Bilt it's the 1826th entry, or january 2nd 1906.

In [4]:
#Here we find the first possible data entry to calculate the FFMC, ISI

def is_float(string):
    try: 
        float(string)
        return True
    except ValueError:
        return False

def first_data_entry(data):
    i=0
    while not(is_float(data[i][2]) & is_float(data[i][3]) & is_float(data[i][4]) & is_float(data[i][5])):
        i+=1
    return i

first=(first_data_entry(data))
    
print(first)    

1826


In [5]:
length=len(data)-first
print(length)

43409


Here we first create te list of FFMCs we calculate this with changed units, that have to be put in the Fire weather index(within value of the code). The initial(0th) entry is 85, because this calculation is an iterative process and has to have a starting point. After 4 or 5 interations the calculations are getting right. 

In [6]:
i=first+1
list=[]
ffmc_calc=[[0,85]]

while i<len(data):
    list=[]
    list.append(data[i][1])
    if not is_float(data[i][4]):
        data[i][4]=0
    if not is_float(data[i][2]):
        data[i][2]=0
    value=FWICLASS(float(data[i][3])/10,float(data[i][5]),float(data[i][2])/10*3.6,float(data[i][4])/10)
    ffmc=value.FFMCcalc(float(ffmc_calc[i-first-1][1]))
    list.append(ffmc)
    ffmc_calc.append(list)
    i+=1

In here the DMC is calculated. For this also the month has to be found for the calculation. The initial value here is 6. It depends on the former value. This index is about the drought of the layers in the ground, one have to check when the first day is of the first year to know if these values are usable. If it's early it's okay to use, but if it's later in the year(let's say june or july) the initial values of these index are not accurate. The second year is definitely useable.

In [7]:
i=first+1
list=[]
dmc_calc=[[0,6]]
while i<len(data):
    list=[]
    list.append(data[i][1])
    value=FWICLASS(float(data[i][3])/10,float(data[i][5]),float(data[i][2])/10*3.6,float(data[i][4])/10)
    month=data[i][1][4:6]
    dmc=value.DMCcalc(float(dmc_calc[i-first-1][1]),mth = int(month))
    list.append(dmc)
    dmc_calc.append(list)
    i+=1

Here the ISI is calculated, ISI is dependant on the FFMC of that day

In [8]:
i=first+1
list=[]
isi_calc=[]
while i<len(data):
    list=[]
    list.append(data[i][1])
    value=FWICLASS(float(data[i][3])/10,float(data[i][5]),float(data[i][2])/10*3.6,float(data[i][4])/10)
    isi=value.ISIcalc(float(ffmc_calc[i-first][1]))
    list.append(isi)
    isi_calc.append(list)
    i+=1

In here DC is calculated, the starting point is 15 and this is dependant on the month and day. Because this index is dependant on the last day and about the drought of the deeper layers, one have to check when the first day is of the first year to know if these values are usable. If it's early it's okay to use, but if it's later in the year(let's say june or july) the initial values of these index are not accurate.

In [9]:
i=first+1
list=[]
dc_calc=[[0,15]]
while i<len(data):
    list=[]
    list.append(data[i][1])
    value=FWICLASS(float(data[i][3])/10,float(data[i][5]),float(data[i][2])/10*3.6,float(data[i][4])/10)
    month=data[i][1][4:6]
    dc=value.DCcalc(float(dc_calc[i-first-1][1]),mth = int(month))
    list.append(dc)
    dc_calc.append(list)
    i+=1

Here BUI is calculated where DMC and DC of that day are used.

In [10]:
i=first+1
list=[]
bui_calc=[]
while i<len(data):
    list=[]
    list.append(data[i][1])
    value=FWICLASS(float(data[i][3])/10,float(data[i][5]),float(data[i][2])/10*3.6,float(data[i][4])/10)
    bui=value.BUIcalc(float(dmc_calc[i-first][1]),dc_calc[i-first][1])
    list.append(bui)
    bui_calc.append(list)
    i+=1

Here FWI is calculated where BUI and ISI are used.

In [11]:
i=first+1
list=[]
fwi_calc=[]
while i<len(data):
    list=[]
    list.append(data[i][1])
    value=FWICLASS(float(data[i][3])/10,float(data[i][5]),float(data[i][2])/10*3.6,float(data[i][4])/10)
    fwi=value.FWIcalc(float(isi_calc[i-first-1][1]),bui_calc[i-first-1][1])
    list.append(fwi)
    fwi_calc.append(list)
    i+=1

Here the dataframes of the different indices are merged into one, where date is in the YYYYMMDD format.

In [12]:
df_ffmc=pd.DataFrame(ffmc_calc)
df_isi=pd.DataFrame(isi_calc)
result = pd.merge(df_ffmc, df_isi, on=0, how='right', suffixes=('_ffmc', '_isi'))
df_dc=pd.DataFrame(dc_calc)
result = pd.merge(result, df_dc, on=0, how='right')
df_dmc=pd.DataFrame(dmc_calc)
result = pd.merge(result, df_dmc, on=0, how='right', suffixes=('_dc', '_dmc'))
df_bui=pd.DataFrame(bui_calc)
result = pd.merge(result, df_bui, on=0, how='right')
df_fwi=pd.DataFrame(fwi_calc)
result = pd.merge(result, df_fwi, on=0, how='right', suffixes=('_bui', '_fwi'))

result.columns = ['date', 'FFMC', 'ISI', 'DC', 'DMC', 'BUI','FWI']

print(result)



           date       FFMC        ISI          DC       DMC       BUI  \
0      19060102  84.220644  12.278793   15.000000  6.014404  6.013288   
1      19060103  82.425889   6.071271   15.532000  6.175801  6.194245   
2      19060104  44.627377   0.286477   13.175577  3.612604  4.286752   
3      19060105  18.412167   0.000257    1.324000  1.257584  0.882536   
4      19060106  34.465255   0.076108    1.666000  1.000000  0.815724   
...         ...        ...        ...         ...       ...       ...   
43403  20241101  45.375846   0.132245  200.962223  1.454636  2.857561   
43404  20241102  55.746225   0.436468  203.006223  1.697655  3.325780   
43405  20241103  60.564315   0.517339  205.122223  1.870672  3.657944   
43406  20241104  66.357422   0.889677  207.292223  2.086512  4.070592   
43407  20241105  68.554471   0.756394  209.102223  2.222274  4.329516   

            FWI  
0      9.717494  
1      5.166888  
2      0.115513  
3      0.000066  
4      0.019262  
...         ...

Here the dataframe is saved in a file FWI_stationnumber.txt file which is of the format csv. For example de Bilt is 260. A place to find an overview of these numbers with the station names is findable here also: https://daggegevens.knmi.nl/

In [13]:
result.to_csv('FWI_'+str(station_code)+'.txt', sep =',', index=False)


